In [1]:
import pandas as pd
import numpy as np
import feather
from sklearn import preprocessing as pe
from tqdm import tqdm

from utils import text_to_wordlist, get_fp
from sklearn.model_selection import train_test_split

In [2]:
def get_wt(cls):
    tot_neg_instances = df_train.shape[0] - p[cls]
    tot_pos_instances = p[cls]
    return float(tot_neg_instances/tot_pos_instances)

In [3]:
D = 2 ** 24

In [4]:
def do_feature_engg(df_all):
    
    # get the share of the gene expression in the corresponding text in the row
    df_all['Gene_Share'] = df_all.apply(lambda row: sum([1 for w in row['Gene'].split(' ') 
                                                       if w in row['Text'].split(' ')]), axis=1)


    # get the share of the variation in the corresponding text in the row
    df_all['Variation_Share'] = df_all.apply(lambda row: sum([1 for w in row['Variation'].split(' ') 
                                                            if w in row['Text'].split(' ')]), axis=1)


    # get the gene length
    df_all['GL'] = df_all['Gene'].apply(lambda x: len(x))

    # get the variation length
    df_all['VL'] = df_all['Variation'].apply(lambda x: len(x))

    # print(sorted(df_all['VL'].values))

    max_len=55 # VL has max length 55 
    for i in range(max_len+1):
        df_all['Gene_'+str(i)] = df_all['Gene'].map(lambda x: str(x[i]) if len(x)>i else '')
        df_all['Variation_'+str(i)] = df_all['Variation'].map(lambda x: str(x[i]) if len(x)>i else '')

    # from https://www.kaggle.com/the1owl/redefining-treatment-0-57456
#     gene_var_lst = sorted(list(df_train.Gene.unique()) + list(df_train.Variation.unique()))
    gene_var_lst = sorted(list(df_all.Gene.unique()) + list(df_all.Variation.unique()))
    gene_var_lst = [x for x in gene_var_lst if len(x.split(' '))==1]

    i_ = 0

    for el in tqdm(gene_var_lst):
        df_all['GV_'+str(el)] = df_all['Text'].map(lambda x: str(x).count(str(el)))
        i_ += 1
    
    # gene expression value counts
    s = df_all['Gene'].value_counts()
    df_all['G_VC'] = df_all['Gene'].apply(lambda gene: s[str(gene)])
    # print(df_all.G_VC[df_all.Gene == 'BRCA1'].count())

    # variation value counts
    s = df_all['Variation'].value_counts()
    df_all['V_VC'] = df_all['Variation'].apply(lambda var: s[str(var)])


    # label encoding, length of column and number of words in the column
    for c in tqdm(df_all.columns):
        if df_all[c].dtype == 'object':
            if c in ['Gene','Variation']:
                lbl = pe.LabelEncoder()
                df_all[c+'_lbl_enc'] = lbl.fit_transform(df_all[c].values)  
                df_all[c+'_len'] = df_all[c].map(lambda x: len(str(x)))
                df_all[c+'_words'] = df_all[c].map(lambda x: len(str(x).split(' ')))
            elif c != 'Text':
                lbl = pe.LabelEncoder()
                df_all[c] = lbl.fit_transform(df_all[c].values)
            if c=='Text': 
                df_all[c+'_len'] = df_all[c].map(lambda x: len(str(x)))
                df_all[c+'_words'] = df_all[c].map(lambda x: len(str(x).split(' '))) 

    return df_all


In [5]:
df_train = feather.read_dataframe('../cache/train1_stage2.feather')

In [6]:
df_train.head()

,Class,Gene,ID,Text,Variation,rsid
0,1,FAM58A,0,Cyclin-dependent kinases (CDKs) regulate a var...,Truncating Mutations,RS#:768583659
1,2,CBL,1,Abstract Background Non-small cell lung canc...,W802*,RS#:387906664
2,2,CBL,1,Abstract Background Non-small cell lung canc...,W802*,RS#:2227988
3,2,CBL,1,Abstract Background Non-small cell lung canc...,W802*,RS#:538190997
4,2,CBL,1,Abstract Background Non-small cell lung canc...,W802*,RS#:121434568


In [7]:
df_test = feather.read_dataframe('../cache/test1_stage2.feather')

In [8]:
df_test.head()

,ID,Gene,Variation,Text,rsid
0,1,CHEK2,H371Y,The incidence of breast cancer is increasing i...,RS#:17886242
1,2,AXIN2,Truncating Mutations,An unselected series of 310 colorectal carcino...,RS#:749846538
2,3,WNT4,E216G,Mycosis fungoides and Sézary syndrome are prim...,None
3,4,SUCLA2,G118R,Regulated progression through the cell cycle ...,RS#:371769427
4,5,BRAF,T599insTT,Pilocytic astrocytoma (PA) is emerging as a tu...,RS#:113488022


In [9]:
y = df_train['Class'].values
df_train = df_train.drop(['Class'], axis=1)

test_ids = df_test['ID'].values

In [10]:
df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)

In [11]:
df_all.isnull().sum()

Gene         0
ID           0
Text         0
Variation    0
rsid         0
dtype: int64

In [12]:
df_all.shape

(23030, 5)

In [13]:
df_all = do_feature_engg(df_all)

100%|██████████| 4259/4259 [00:33<00:00, 125.41it/s]


In [14]:
df_train = df_all.iloc[:len(df_train)]
df_test = df_all.iloc[len(df_train):]

In [15]:
df_train['Class'] = y

/home/watts/anaconda2/envs/aind-dog/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [16]:
df_train.shape

(22044, 4268)

In [17]:
df_test.shape

(986, 4267)

In [18]:
df_train.head()

,Gene,ID,Text,Variation,rsid,Gene_Share,Variation_Share,GL,VL,Gene_0,...,V_VC,Gene_lbl_enc,Gene_len,Gene_words,Text_len,Text_words,Variation_lbl_enc,Variation_len,Variation_words,Class
0,FAM58A,0,Cyclin-dependent kinases (CDKs) regulate a var...,Truncating Mutations,1656,1,1,6,20,5,...,390,127,6,1,39672,6105,3457,20,2,1
1,CBL,1,Abstract Background Non-small cell lung canc...,W802*,931,1,1,3,5,2,...,6,58,3,1,36691,5783,3745,5,1,2
2,CBL,1,Abstract Background Non-small cell lung canc...,W802*,675,1,1,3,5,2,...,6,58,3,1,36691,5783,3745,5,1,2
3,CBL,1,Abstract Background Non-small cell lung canc...,W802*,1130,1,1,3,5,2,...,6,58,3,1,36691,5783,3745,5,1,2
4,CBL,1,Abstract Background Non-small cell lung canc...,W802*,135,1,1,3,5,2,...,6,58,3,1,36691,5783,3745,5,1,2


In [19]:
df_test.head()

,Gene,ID,Text,Variation,rsid,Gene_Share,Variation_Share,GL,VL,Gene_0,...,G_VC,V_VC,Gene_lbl_enc,Gene_len,Gene_words,Text_len,Text_words,Variation_lbl_enc,Variation_len,Variation_words
22044,CHEK2,1,The incidence of breast cancer is increasing i...,H371Y,558,1,1,5,5,2,...,55,22,74,5,1,33403,4991,1378,5,1
22045,AXIN2,2,An unselected series of 310 colorectal carcino...,Truncating Mutations,1465,1,1,5,20,0,...,14,390,34,5,1,66400,10348,3457,20,2
22046,WNT4,3,Mycosis fungoides and Sézary syndrome are prim...,E216G,0,0,1,4,5,21,...,4,1,392,4,1,58544,8638,672,5,1
22047,SUCLA2,4,Regulated progression through the cell cycle ...,G118R,858,0,1,6,5,17,...,2,1,364,6,1,42023,6221,1038,5,1
22048,BRAF,5,Pilocytic astrocytoma (PA) is emerging as a tu...,T599insTT,60,1,0,4,9,1,...,469,2,49,4,1,22499,3280,3399,9,1


In [20]:
feather.write_dataframe(df_train, '../cache/train1_stage2_fe.feather')

In [21]:
feather.write_dataframe(df_test, '../cache/test1_stage2_fe.feather')

In [22]:
df_train = feather.read_dataframe('../cache/train1_stage2_fe.feather')

In [23]:
df_test = feather.read_dataframe('../cache/test1_stage2_fe.feather')

In [24]:
df_train.shape

(22044, 4268)

In [25]:
df_train['Text'] = [text_to_wordlist(w) for w in df_train['Text'].values]
df_test['Text'] = [text_to_wordlist(w) for w in df_test['Text'].values]

In [26]:
test_id = df_test.ID.values
df = pd.DataFrame()
df['ID'] = test_id
df.to_csv('../cache/stage2_test_id.csv', index=False)

In [27]:
y = df_train.Class.values
df = pd.DataFrame()
df['y'] = y
df.to_csv('../cache/stage2_labels.csv', index=False)

In [28]:
df_train = df_train.drop('Class',axis=1)

In [29]:
rsid = df_train.rsid.values

In [30]:
df_train.shape

(22044, 4267)

In [31]:
sum(df_train.isnull().any() == True)

0

In [32]:
np.isfinite([1,2,3])

array([ True,  True,  True], dtype=bool)

In [33]:
df_train.head()

,Gene,ID,Text,Variation,rsid,Gene_Share,Variation_Share,GL,VL,Gene_0,...,G_VC,V_VC,Gene_lbl_enc,Gene_len,Gene_words,Text_len,Text_words,Variation_lbl_enc,Variation_len,Variation_words
0,FAM58A,0,cyclin - depend kinas cdks regul varieti funda...,Truncating Mutations,1656,1,1,6,20,5,...,1,390,127,6,1,39672,6105,3457,20,2
1,CBL,1,abstract background non - small cell lung canc...,W802*,931,1,1,3,5,2,...,153,6,58,3,1,36691,5783,3745,5,1
2,CBL,1,abstract background non - small cell lung canc...,W802*,675,1,1,3,5,2,...,153,6,58,3,1,36691,5783,3745,5,1
3,CBL,1,abstract background non - small cell lung canc...,W802*,1130,1,1,3,5,2,...,153,6,58,3,1,36691,5783,3745,5,1
4,CBL,1,abstract background non - small cell lung canc...,W802*,135,1,1,3,5,2,...,153,6,58,3,1,36691,5783,3745,5,1


In [34]:
p = df_train['Gene_Share'].values

In [35]:
sum(np.isfinite(p))

22044

In [36]:
t_col = ['Gene', 'Variation', 'Text']

In [37]:
l = [np.isfinite(df_train[c].values) for c in df_train.columns if c not in t_col]

In [38]:
sum(sum(l) != 4326)

22044

In [39]:
l = [np.isinf(df_train[c].values) for c in df_train.columns if c not in t_col]

In [40]:
sum(sum(l) != 0)

0

In [41]:
df_train = df_train.replace([np.inf, -np.inf], np.nan)

In [42]:
sum(df_train.isnull().any() == True)

0

In [43]:
fp = 0
fp = get_fp(fp)

In [44]:
tr = df_train

In [45]:
tr = tr.drop(t_col, axis=1)

In [46]:
sum(np.isfinite(tr).sum() != 22044)

0

In [47]:
df_train['rsid'] = rsid

In [48]:
pd.set_option('display.max_columns', None)

In [49]:
df_train = df_train.loc[:, (df_train != 0).any(axis=0)] # drop columns that are only zero

In [50]:
df_train.head()

Gene  ID                                               Text  \
0  FAM58A   0  cyclin - depend kinas cdks regul varieti funda...   
1     CBL   1  abstract background non - small cell lung canc...   
2     CBL   1  abstract background non - small cell lung canc...   
3     CBL   1  abstract background non - small cell lung canc...   
4     CBL   1  abstract background non - small cell lung canc...   

              Variation  rsid  Gene_Share  Variation_Share  GL  VL  Gene_0  \
0  Truncating Mutations  1656           1                1   6  20       5   
1                 W802*   931           1                1   3   5       2   
2                 W802*   675           1                1   3   5       2   
3                 W802*  1130           1                1   3   5       2   
4                 W802*   135           1                1   3   5       2   

   Variation_0  Gene_1  Variation_1  Gene_2  Variation_2  Gene_3  Variation_3  \
0           24       3           40      18           41       7           45   
1           26       4            9      17            2       0            6   
2           26       4            9      17            2       0            6   
3           26       4            9      17            2       0            6   
4           26       4            9      17            2       0            6   

   Gene_4  Variation_4  Gene_5  Variation_5  Gene_6  Variation_6  Gene_7  \
0      10           40       9           41       0           47       0   
1       0            2       0            0       0            0       0   
2       0            2       0            0       0            0       0   
3       0            2       0            0       0            0       0   
4       0            2       0            0       0            0       0   

   Variation_7  Variation_8  Variation_9  Variation_10  Variation_11  \
0           41           44           32             1            13   
1            0            0            0             0             0   
2            0            0            0             0             0   
3            0            0            0             0             0   
4            0            0            0             0             0   

   Variation_12  Variation_13  Variation_14  Variation_15  Variation_16  \
0            32            29            17            27            18   
1             0             0             0             0             0   
2             0             0             0             0             0   
3             0             0             0             0             0   
4             0             0             0             0             0   

   Variation_17  Variation_18  Variation_19  Variation_20  Variation_21  \
0            19            15            19             0             0   
1             0             0             0             0             0   
2             0             0             0             0             0   
3             0             0             0             0             0   
4             0             0             0             0             0   

   Variation_22  Variation_23  Variation_24  Variation_25  Variation_26  \
0             0             0             0             0             0   
1             0             0             0             0             0   
2             0             0             0             0             0   
3             0             0             0             0             0   
4             0             0             0             0             0   

   Variation_27  Variation_28  Variation_29  Variation_30  Variation_31  \
0             0             0             0             0             0   
1             0             0             0             0             0   
2             0             0             0             0             0   
3             0             0             0             0             0   
4             0       

In [51]:
df_train.shape

(22044, 3006)

In [52]:
tr_cols = df_train.columns

In [53]:
df_test = df_test[tr_cols]

In [54]:
df_test.shape

(986, 3006)

In [55]:
from sklearn.base import BaseEstimator as be
from sklearn.base import TransformerMixin as tm
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
class pm_numeric_cols(be, tm):
    def fit(self, x, y=None):
        return self
    def transform(self, x):
        x = x.drop(['Gene', 'Variation','ID','Text'],axis=1).values
        return x

class pm_txt_col(be, tm):
    def __init__(self, key):
        self.key = key
    def fit(self, x, y=None):
        return self
    def transform(self, x):
        return x[self.key].apply(str)
fp = Pipeline([
        ('union', FeatureUnion(
            n_jobs = 1,
            transformer_list = [
                ('standard', pm_numeric_cols()),
                ('pi1', Pipeline([('Gene', pm_txt_col('Gene')), 
                                           ('count_Gene', CountVectorizer(analyzer=u'char',ngram_range=(1, 8))), 
                                           ('tsvd1', TruncatedSVD(n_components=8, n_iter=5, random_state=12))])),
                ('pi2', Pipeline([('Variation', pm_txt_col('Variation')), 
                                           ('count_Variation', CountVectorizer(analyzer=u'char',ngram_range=(1, 8))), 
                                           ('tsvd2', TruncatedSVD(n_components=8, n_iter=5, random_state=12))])),     
                ('pi4',Pipeline([('Text', pm_txt_col('Text')), 
                                       ('hv', HashingVectorizer(decode_error='ignore', n_features=2 ** 16,
                                                                non_negative=True, 
                                                                ngram_range=(1, 3))),
                                       ('tfidf_Text', TfidfTransformer()), 
                                       ('tsvd4', TruncatedSVD(n_components=300, n_iter=25, 
                                                                            random_state=12))]))

            ])
        )])


# fp = Pipeline([
#     ('union', FeatureUnion(
#         n_jobs = -1,
#         transformer_list = [
#             ('standard', pm_numeric_cols()),
#             ('pi1', Pipeline([('Gene', pm_txt_col('Gene')), 
#                                        ('count_Gene', CountVectorizer(analyzer=u'char', ngram_range=(1, 8))), 
#                                        ('tsvd1', TruncatedSVD(n_components=20, n_iter=5, random_state=12))])),
#             ('pi2', Pipeline([('Variation', pm_txt_col('Variation')), 
#                                        ('count_Variation', CountVectorizer(analyzer=u'char', ngram_range=(1, 8))), 
#                                        ('tsvd2',TruncatedSVD(n_components=20, n_iter=5, random_state=12))])),
#             #commented for Kaggle Limits
# #             ('pi3', pipeline.Pipeline([('Text', cust_txt_col('Text')), 
# #                                        ('hv', feature_extraction.text.HashingVectorizer(decode_error='ignore', n_features=2 ** 16, non_negative=True, ngram_range=(1, 3))),
# #                                        ('tfidf_Text', feature_extraction.text.TfidfTransformer()), 
# #                                        ('tsvd3', decomposition.TruncatedSVD(n_components=300, n_iter=25, random_state=12))]))

        
#         ])
#     )])

In [56]:
22044/11

2004.0

In [59]:
train_t = np.empty([0, 3318])
print(train_t.shape)
step = 2004
for i in range(0, len(y), step):
    print(i)
    step_end = i+step
    step_end = step_end if step_end < len(y) else len(y)
    _train = fp.fit_transform(df_train.iloc[i:step_end])
    train_t = np.vstack((train_t, _train))
df_train = train_t
print(df_train.shape)

(0, 3318)
0


/home/watts/anaconda2/envs/aind-dog/lib/python3.6/site-packages/sklearn/feature_extraction/hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)
/home/watts/anaconda2/envs/aind-dog/lib/python3.6/site-packages/sklearn/feature_extraction/hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)


2004


/home/watts/anaconda2/envs/aind-dog/lib/python3.6/site-packages/sklearn/feature_extraction/hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)
/home/watts/anaconda2/envs/aind-dog/lib/python3.6/site-packages/sklearn/feature_extraction/hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)


4008


...

18036


/home/watts/anaconda2/envs/aind-dog/lib/python3.6/site-packages/sklearn/feature_extraction/hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)
/home/watts/anaconda2/envs/aind-dog/lib/python3.6/site-packages/sklearn/feature_extraction/hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)


20040


/home/watts/anaconda2/envs/aind-dog/lib/python3.6/site-packages/sklearn/feature_extraction/hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)
/home/watts/anaconda2/envs/aind-dog/lib/python3.6/site-packages/sklearn/feature_extraction/hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)


(22044, 3318)


In [60]:
print(i)

20040


In [61]:
print(_train.shape)

(2004, 3318)


In [62]:
df_test.shape

(986, 3006)

In [67]:
test_t = np.empty([0, 3318])
step = 493
for i in range(0, len(test_id), step):
    print(i)
    step_end = i+step
    step_end = step_end if step_end < len(test_id) else len(test_id)
    _test = fp.fit_transform(df_test.iloc[i:step_end])
    test_t = np.vstack((test_t, _test))
df_test = test_t
print(df_test.shape)

0


/home/watts/anaconda2/envs/aind-dog/lib/python3.6/site-packages/sklearn/feature_extraction/hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)
/home/watts/anaconda2/envs/aind-dog/lib/python3.6/site-packages/sklearn/feature_extraction/hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)


493


/home/watts/anaconda2/envs/aind-dog/lib/python3.6/site-packages/sklearn/feature_extraction/hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)
/home/watts/anaconda2/envs/aind-dog/lib/python3.6/site-packages/sklearn/feature_extraction/hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)


(986, 3318)


In [89]:
_test.shape

(493, 3318)

In [ ]:
# df_train = fp.fit_transform(df_train)
# print (df_train.shape)

In [ ]:
# df_test = fp.fit_transform(df_test)
# print (df_test.shape)

In [90]:
np.save('../cache/train_stage2_fe2', df_train)
np.save('../cache/test_stage2_fe2', df_test)

In [69]:
# now claculate weights

In [91]:
df_train = feather.read_dataframe('../cache/train_stage2_fe.feather')

In [92]:
df_train.head()

Gene  ID                                               Text  \
0  FAM58A   0  Cyclin-dependent kinases (CDKs) regulate a var...   
1     CBL   1   Abstract Background  Non-small cell lung canc...   
2     CBL   2   Abstract Background  Non-small cell lung canc...   
3     CBL   3  Recent evidence has demonstrated that acquired...   
4     CBL   4  Oncogenic mutations in the monomeric Casitas B...   

              Variation                      GeneVar  Gene_Share  \
0  Truncating Mutations  FAM58A Truncating Mutations           1   
1                 W802*                    CBL W802*           1   
2                 Q249E                    CBL Q249E           1   
3                 N454D                    CBL N454D           1   
4                 L399V                    CBL L399V           1   

   Variation_Share  Gene_Variation_Share  GL  VL  Gene_0  Variation_0  \
0                1                     2   6  20       5           24   
1                1                     2   3   5       2           26   
2                1                     2   3   5       2           21   
3                1                     2   3   5       2           18   
4                1                     2   3   5       2           16   

   GeneVar_0  Gene_1  Variation_1  GeneVar_1  Gene_2  Variation_2  GeneVar_2  \
0          5       3           40          3      18           41         18   
1          2       4            9          4      17            2         17   
2          2       4            3          4      17            6         17   
3          2       4            5          4      17            7         17   
4          2       4            4          4      17           11         17   

   Gene_3  Variation_3  GeneVar_3  Gene_4  Variation_4  GeneVar_4  Gene_5  \
0       7           45          7      10           40         10       9   
1       0            6          0       0            2         32       0   
2       0           13          0       0           18         27       0   
3       0            8          0       0           17         24       0   
4       0           13          0       0           33         22       0   

   Variation_5  GeneVar_5  Gene_6  Variation_6  GeneVar_6  Gene_7  \
0           41         12       0           47          0       0   
1            0         10       0            0          1       0   
2            0          4       0            0          5       0   
3            0          6       0            0          6       0   
4            0          5       0            0         10       0   

   Variation_7  GeneVar_7  Gene_8  Variation_8  GeneVar_8  Gene_9  \
0           41         32       0           44         50       0   
1            0          6       0            0          2       0   
2            0         13       0            0         18       0   
3            0          8       0            0         17       0   
4            0         13       0            0         33       0   

   Variation_9  GeneVar_9  Gene_10  Variation_10  GeneVar_10  Gene_11  \
0           32         55        0             1          46        0   
1            0          0        0             0           0        0   
2            0          0        0             0           0        0   
3            0          0        0             0           0        0   
4            0          0        0             0           0        0   

   Variation_11  GeneVar_11  Gene_12  Variation_12  GeneVar_12  Gene_13  \
0            13          42        0            32          39        0   
1             0           0        0             0           0        0   
2             0           0        0             0           0        0   
3             0           0        0             0           0        0   
4             0           0        0             0           0        0   

   Variation_13  GeneVar_13  Gene_14  Variation_14  GeneVar_14  Gene_15  \
0        

In [93]:
df_train.shape

(3689, 4335)

In [94]:
pd.value_counts(df_train['Class'])

7    1054
4     751
1     662
2     498
6     297
5     267
3      96
9      43
8      21
Name: Class, dtype: int64

In [95]:
p =pd.value_counts(df_train['Class'])

In [96]:
print(p[7])

1054


In [97]:
print(p[8])

21


In [98]:
df_train['wt'] =  df_train['Class'].map(lambda s: get_wt(s))

In [99]:
df_train.head()

Gene  ID                                               Text  \
0  FAM58A   0  Cyclin-dependent kinases (CDKs) regulate a var...   
1     CBL   1   Abstract Background  Non-small cell lung canc...   
2     CBL   2   Abstract Background  Non-small cell lung canc...   
3     CBL   3  Recent evidence has demonstrated that acquired...   
4     CBL   4  Oncogenic mutations in the monomeric Casitas B...   

              Variation                      GeneVar  Gene_Share  \
0  Truncating Mutations  FAM58A Truncating Mutations           1   
1                 W802*                    CBL W802*           1   
2                 Q249E                    CBL Q249E           1   
3                 N454D                    CBL N454D           1   
4                 L399V                    CBL L399V           1   

   Variation_Share  Gene_Variation_Share  GL  VL  Gene_0  Variation_0  \
0                1                     2   6  20       5           24   
1                1                     2   3   5       2           26   
2                1                     2   3   5       2           21   
3                1                     2   3   5       2           18   
4                1                     2   3   5       2           16   

   GeneVar_0  Gene_1  Variation_1  GeneVar_1  Gene_2  Variation_2  GeneVar_2  \
0          5       3           40          3      18           41         18   
1          2       4            9          4      17            2         17   
2          2       4            3          4      17            6         17   
3          2       4            5          4      17            7         17   
4          2       4            4          4      17           11         17   

   Gene_3  Variation_3  GeneVar_3  Gene_4  Variation_4  GeneVar_4  Gene_5  \
0       7           45          7      10           40         10       9   
1       0            6          0       0            2         32       0   
2       0           13          0       0           18         27       0   
3       0            8          0       0           17         24       0   
4       0           13          0       0           33         22       0   

   Variation_5  GeneVar_5  Gene_6  Variation_6  GeneVar_6  Gene_7  \
0           41         12       0           47          0       0   
1            0         10       0            0          1       0   
2            0          4       0            0          5       0   
3            0          6       0            0          6       0   
4            0          5       0            0         10       0   

   Variation_7  GeneVar_7  Gene_8  Variation_8  GeneVar_8  Gene_9  \
0           41         32       0           44         50       0   
1            0          6       0            0          2       0   
2            0         13       0            0         18       0   
3            0          8       0            0         17       0   
4            0         13       0            0         33       0   

   Variation_9  GeneVar_9  Gene_10  Variation_10  GeneVar_10  Gene_11  \
0           32         55        0             1          46        0   
1            0          0        0             0           0        0   
2            0          0        0             0           0        0   
3            0          0        0             0           0        0   
4            0          0        0             0           0        0   

   Variation_11  GeneVar_11  Gene_12  Variation_12  GeneVar_12  Gene_13  \
0            13          42        0            32          39        0   
1             0           0        0             0           0        0   
2             0           0        0             0           0        0   
3             0           0        0             0           0        0   
4             0           0        0             0           0        0   

   Variation_13  GeneVar_13  Gene_14  Variation_14  GeneVar_14  Gene_15  \
0        

In [100]:
wt = df_train.wt.values
df = pd.DataFrame()
df['wt'] = wt
df.to_csv('../cache/stage2_weights.csv', index=False)

In [101]:
np.save('../cache/stage2_train_weights', wt)

In [102]:
my_wt = {}
n_class = 9

In [103]:
for cls in range(n_class):
    my_wt[cls+1] = get_wt(cls+1)

In [104]:
my_wt

{1: 4.572507552870091,
 2: 6.407630522088353,
 3: 37.427083333333336,
 4: 3.912117177097204,
 5: 12.816479400749063,
 6: 11.42087542087542,
 7: 2.5,
 8: 174.66666666666666,
 9: 84.79069767441861}

In [105]:
np.save('../cache/stage2_train_weights_per_class', my_wt)

In [106]:
# now create validation data to be used by all stage2 classifers

In [107]:
df_train = np.load('../cache/train_stage2_fe2.npy')
df1 = pd.read_csv('../cache/stage2_labels.csv')
y = df1['y'].values

In [108]:
x1,x2,y1,y2 = train_test_split(df_train, y, test_size=0.2, random_state=42, stratify=y)

In [109]:
np.save('../cache/train_stage2_x1', x1)
np.save('../cache/train_stage2_x2', x2)
np.save('../cache/train_stage2_y1', y1)
np.save('../cache/train_stage2_y2', y2)

In [110]:
p =pd.value_counts(y1)
df_train = pd.DataFrame(x1)
my_wt = {}
n_class = 9

for cls in range(n_class):
    my_wt[cls+1] = get_wt(cls+1)
print(my_wt)

np.save('../cache/stage2_x1_weights_per_class', my_wt)

{1: 5.144599303135888, 2: 7.956323006602336, 3: 25.923664122137403, 4: 2.7056104223576383, 5: 9.276806526806526, 6: 10.147281921618205, 7: 3.5840914998700284, 8: 206.47058823529412, 9: 115.01973684210526}


In [111]:
df_train = pd.DataFrame(x2)
p =pd.value_counts(y2)

my_wt = {}
n_class = 9

for cls in range(n_class):
    my_wt[cls+1] = get_wt(cls+1)
print(my_wt)

{1: 5.14066852367688, 2: 7.961382113821138, 3: 26.049079754601227, 4: 2.705042016806723, 5: 9.277389277389277, 6: 10.133838383838384, 7: 3.583160083160083, 8: 208.95238095238096, 9: 115.02631578947368}


In [112]:
np.save('../cache/stage2_x2_weights_per_class', my_wt)